In [ ]:
%matplotlib inline

In [ ]:
import gc
import pathlib
import sys

In [ ]:
import joblib
import pandas as pd
from catboost import CatBoostRegressor
from catboost import Pool

In [ ]:
src_dir = "../../src"

In [ ]:
sys.path.append(src_dir)

In [ ]:
from package.constants import *

In [ ]:
params = {
    "eval_metric": "RMSE",
    "max_bin": 32,
    "objective": "Poisson",
    "random_state": random_state,
    "thread_count": -1,
    "verbose": 10,
    # See https://catboost.ai/docs/concepts/faq.html#overcome-out-of-memory
    "task_type": "GPU",
    "boosting_type": "Plain",
    "max_ctr_complexity": 1,
}

In [ ]:
train = pd.read_parquet(train_path)

In [ ]:
# See https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/135896
is_train = (train["date"] >= "2014-03-28") & (train["date"] < validation_start_date)
is_valid = (train["date"] >= validation_start_date) & (train["date"] < test_start_date)

In [ ]:
is_train |= is_valid
# See https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/138268
is_train &= train["sell_price"].notnull()

In [ ]:
train_pool = Pool(
    train.loc[is_train, features],
    train.loc[is_train, transformed_target],
    cat_features=categorical_features,
)

In [ ]:
del train
del is_train
del is_valid

In [ ]:
gc.collect()

In [ ]:
model = CatBoostRegressor(**params)

In [ ]:
%%time
model.fit(train_pool)

In [ ]:
joblib.dump(model, cb_reg_path)